In [71]:
import pandas as pd
import numpy as np
import yfinance as yf
import os.path
from sklearn.preprocessing import MinMaxScaler

### Reading file

In [12]:
df = pd.read_csv('../data/data_1_esg_raw.csv') 
df.head(5)

,company_name,ticker_symbol,company_esg_score,company_esg_score_group,industry
0,HENSOLDT AG,ETR:HAG,14.9,Low ESG Risk,Aerospace & Defense
1,Aptiv Plc,NYS:APTV,9.1,Negligible ESG Risk,Auto Components
2,"BorgWarner, Inc.",NYS:BWA,10.0,Negligible ESG Risk,Auto Components
3,Bosch Fren Sistemleri Sanayi ve Ticaret AS,IST:BFREN.E,8.1,Negligible ESG Risk,Auto Components
4,Bosch Ltd.,BOM:500530,6.5,Negligible ESG Risk,Auto Components


### Sort by ESG score

In [13]:
df_sorted = df.sort_values('company_esg_score')
df_sorted

,company_name,ticker_symbol,company_esg_score,company_esg_score_group,industry
4184,ROADIS Transportation Holding SLU,-,4.0,Negligible ESG Risk,Transportation Infrastructure
1018,JAB Holding Co. SARL,-,4.2,Negligible ESG Risk,Diversified Financials
2037,Dexus,ASX:DXS,4.2,Negligible ESG Risk,Real Estate
4180,Entidad Pública Empresarial ADIF-Alta Velocidad,-,4.3,Negligible ESG Risk,Transportation Infrastructure
3498,RS Group Plc,LON:RS1,4.5,Negligible ESG Risk,Technology Hardware
...,...,...,...,...,...
1428,International Finance Facility for Immunisatio...,-,20.0,Low ESG Risk,Healthcare
3843,Orange SA,PAR:ORA,20.0,Low ESG Risk,Telecommunication Services
4365,Promigas SA ESP,BOG:PROMIGAS,20.0,Low ESG Risk,Utilities
3950,Li & Fung Ltd.,-,20.0,Low ESG Risk,Textiles & Apparel


In [14]:
len(df_sorted)

4386

### Group by industry and company-esg-score-group

In [15]:
# Group by the first and second columns and count the occurrences
df_grouped = df.groupby(['industry', 'company_esg_score_group']).size().reset_index(name='Count')
df_grouped = df_grouped.sort_values(by=['industry', 'company_esg_score_group'], ascending=[True, False])
df_grouped

,industry,company_esg_score_group,Count
0,Aerospace & Defense,Low ESG Risk,1
2,Auto Components,Negligible ESG Risk,12
1,Auto Components,Low ESG Risk,105
3,Automobiles,Low ESG Risk,22
5,Banks,Negligible ESG Risk,43
...,...,...,...
63,Transportation,Low ESG Risk,107
65,Transportation Infrastructure,Negligible ESG Risk,21
64,Transportation Infrastructure,Low ESG Risk,92
67,Utilities,Negligible ESG Risk,11


### Remove stocks without a valid stock exchange

In [16]:
len(df_sorted[df_sorted['ticker_symbol'] != '-'])

3801

In [17]:
df_sorted[df_sorted['ticker_symbol'] != '-']

,company_name,ticker_symbol,company_esg_score,company_esg_score_group,industry
2037,Dexus,ASX:DXS,4.2,Negligible ESG Risk,Real Estate
3498,RS Group Plc,LON:RS1,4.5,Negligible ESG Risk,Technology Hardware
3486,"Kimball Electronics, Inc.",NAS:KE,4.5,Negligible ESG Risk,Technology Hardware
2100,TAG Immobilien AG,ETR:TEG,4.6,Negligible ESG Risk,Real Estate
2105,Unibail-Rodamco-Westfield SE,PAR:URW,4.7,Negligible ESG Risk,Real Estate
...,...,...,...,...,...
1140,Nippon Life India Asset Management Ltd.,BOM:540767,20.0,Low ESG Risk,Diversified Financials
3850,"Taiwan Mobile Co., Ltd.",TAI:3045,20.0,Low ESG Risk,Telecommunication Services
4160,Union Pacific Corp.,NYS:UNP,20.0,Low ESG Risk,Transportation
3843,Orange SA,PAR:ORA,20.0,Low ESG Risk,Telecommunication Services


### Create new columns "stock_exchange" & "stock_ticker_symbol"

In [18]:
# Create new column stock_exchange by splitting ticker_symbol
df_sorted['stock_exchange'] = df_sorted['ticker_symbol'].str.split(':').str[0]

In [19]:
# Create new column stock_ticker_symbol by splitting ticker_symbol
df_sorted['stock_ticker_symbol'] = df_sorted['ticker_symbol'].str.split(':').str[1]

In [20]:
df_sorted[df_sorted['ticker_symbol'] != '-']['stock_exchange'].value_counts('')

stock_exchange
NYS    551
NAS    408
TKS    372
LON    226
TAI    183
      ... 
LIM      1
LIT      1
BRA      1
FRA      1
BER      1
Name: count, Length: 62, dtype: int64

### Selected Stock-Exchanges

In [21]:
len(df_sorted)

4386

### Filter Stock Exchanges

In [22]:
# drop existing column "stock_ticker_symbol"
df_sorted = df_sorted.drop(columns=['ticker_symbol'])

In [23]:
# Filter by a single column value 
df_filtered = df_sorted[df_sorted['stock_exchange'].isin(['NYS', 'NAS', 'TKS', 'LON', 'ETR'])]
len(df_filtered)

1662

In [24]:
df_filtered

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol
3498,RS Group Plc,4.5,Negligible ESG Risk,Technology Hardware,LON,RS1
3486,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE
2100,TAG Immobilien AG,4.6,Negligible ESG Risk,Real Estate,ETR,TEG
2068,LEG Immobilien SE,5.1,Negligible ESG Risk,Real Estate,ETR,LEG
521,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS
...,...,...,...,...,...,...
3676,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI
2905,Moonpig Group Plc,20.0,Low ESG Risk,Retailing,LON,MOON
1963,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO
3057,"Japan Material Co., Ltd.",20.0,Low ESG Risk,Semiconductors,TKS,6055


#### Group by industry and company-esg-score-group

In [25]:
len(df_filtered)

1662

In [26]:
# Group by the first and second columns and count the occurrences
df_filtered_grouped = df_filtered.groupby(['industry', 'company_esg_score_group']).size().reset_index(name='Count')
df_filtered_grouped = df_filtered_grouped.sort_values(by=['industry', 'company_esg_score_group'], ascending=[True, False])
df_filtered_grouped

,industry,company_esg_score_group,Count
0,Aerospace & Defense,Low ESG Risk,1
2,Auto Components,Negligible ESG Risk,5
1,Auto Components,Low ESG Risk,45
3,Automobiles,Low ESG Risk,4
5,Banks,Negligible ESG Risk,1
4,Banks,Low ESG Risk,15
6,Building Products,Low ESG Risk,13
7,Chemicals,Low ESG Risk,11
9,Commercial Services,Negligible ESG Risk,32
8,Commercial Services,Low ESG Risk,83


In [27]:
df_filtered.to_csv('../data/data_2_esg_filtered.csv')

### Adding market capital of companies

In [28]:
if 'df_market_cap' not in locals():
    df_market_cap = pd.read_csv('../data/data_2_esg_filtered.csv')
df_market_cap

,Unnamed: 0,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol
0,3498,RS Group Plc,4.5,Negligible ESG Risk,Technology Hardware,LON,RS1.L
1,3486,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE
2,2100,TAG Immobilien AG,4.6,Negligible ESG Risk,Real Estate,ETR,TEG.DE
3,2068,LEG Immobilien SE,5.1,Negligible ESG Risk,Real Estate,ETR,LEG.DE
4,521,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS
...,...,...,...,...,...,...,...
1657,3676,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI
1658,2905,Moonpig Group Plc,20.0,Low ESG Risk,Retailing,LON,MOON.L
1659,1963,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO
1660,3057,"Japan Material Co., Ltd.",20.0,Low ESG Risk,Semiconductors,TKS,6055.T


#### Updating `stock_ticker_symbol` for Frankfurt and London
Frankfurt & London tickers require a postfix '.DE' & '.L' respectively for Yahoo API to recognize them.

In [29]:
# Postfix to add
frankfurt_postfix = '.DE'
london_postfix = '.L'
tokyo_postfix = '.T'

# Condition: Add postfix '.DE' to 'stock_exchange' column if the 'stock_ticker_symbol' column is 'ETR' (Frankfurt)
df_market_cap['stock_ticker_symbol'] = np.where((df_market_cap['stock_exchange'] == 'ETR'), df_market_cap['stock_ticker_symbol'] + frankfurt_postfix, df_market_cap['stock_ticker_symbol'])

# Condition: Add postfix '.L' to 'stock_exchange' column if the 'stock_ticker_symbol' column is 'Lon' (London)
df_market_cap['stock_ticker_symbol'] = np.where((df_market_cap['stock_exchange'] == 'LON'), df_market_cap['stock_ticker_symbol'] + london_postfix, df_market_cap['stock_ticker_symbol'])

# Condition: Add postfix '.T' to 'stock_exchange' column if the 'stock_ticker_symbol' column is 'TKS' (Tokyo)
df_market_cap['stock_ticker_symbol'] = np.where((df_market_cap['stock_exchange'] == 'TKS'), df_market_cap['stock_ticker_symbol'] + tokyo_postfix, df_market_cap['stock_ticker_symbol'])

In [30]:
df_market_cap[['stock_exchange', 'stock_ticker_symbol']]

,stock_exchange,stock_ticker_symbol
0,LON,RS1.L.L
1,NAS,KE
2,ETR,TEG.DE.DE
3,ETR,LEG.DE.DE
4,NYS,SCS
...,...,...
1657,NAS,KVHI
1658,LON,MOON.L.L
1659,NAS,NEO
1660,TKS,6055.T.T


#### Fetch Market-Capital

In [31]:
import time

def fetch_market_cap(tickers, batch_size=100, delay=2):
    market_caps = []

    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i+batch_size]
        tickers_str = ' '.join(batch)

        try:
            data = yf.Tickers(tickers_str)
            for ticker in batch:
                info = data.tickers[ticker].info
                market_cap = info.get('marketCap', 'N/A')
                market_caps.append({'stock_ticker_symbol': ticker, 'market_capital': market_cap})
        except Exception as e:
            print(f"Error fetching data for batch starting at index {i}: {e}")

        time.sleep(delay)  # delay to avoid getting blocked

    return pd.DataFrame(market_caps)


# Fetch market cap in batches of 50 with a 3-second delay between batches
if os.path.isfile('market_caps.csv'):
    df_market_caps = fetch_market_cap(df_market_cap['stock_ticker_symbol'], batch_size=50, delay=3)

print(df_market_caps)


      Unnamed: 0 stock_ticker_symbol  market_capital
0              0               RS1.L    3.723188e+09
1              1                  KE    5.958660e+08
2              2              TEG.DE    2.539239e+09
3              3              LEG.DE    6.009704e+09
4              4                 SCS    1.588731e+09
...          ...                 ...             ...
1657        1657                KVHI    9.148317e+07
1658        1658              MOON.L    7.034287e+08
1659        1659                 NEO    1.979536e+09
1660        1660              6055.T    2.270510e+11
1661        1661                 UNP    1.488027e+11

[1662 rows x 3 columns]


In [32]:
df_market_caps.to_csv('../data/data_3_market_caps.csv')

In [33]:
if 'df_market_caps' not in locals():
    df_market_caps = pd.read_csv('../data/data_3_market_caps.csv')

In [39]:
df_market_caps = df_market_caps[['stock_ticker_symbol', 'market_capital']]
df_market_caps

,stock_ticker_symbol,market_capital
0,RS1.L,3.723188e+09
1,KE,5.958660e+08
2,TEG.DE,2.539239e+09
3,LEG.DE,6.009704e+09
4,SCS,1.588731e+09
...,...,...
1657,KVHI,9.148317e+07
1658,MOON.L,7.034287e+08
1659,NEO,1.979536e+09
1660,6055.T,2.270510e+11


In [44]:
df_market_cap = df_market_cap.loc[:, ~df_market_cap.columns.str.contains('^Unnamed')]

df_market_cap

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol
0,RS Group Plc,4.5,Negligible ESG Risk,Technology Hardware,LON,RS1.L.L
1,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE
2,TAG Immobilien AG,4.6,Negligible ESG Risk,Real Estate,ETR,TEG.DE.DE
3,LEG Immobilien SE,5.1,Negligible ESG Risk,Real Estate,ETR,LEG.DE.DE
4,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS
...,...,...,...,...,...,...
1657,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI
1658,Moonpig Group Plc,20.0,Low ESG Risk,Retailing,LON,MOON.L.L
1659,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO
1660,"Japan Material Co., Ltd.",20.0,Low ESG Risk,Semiconductors,TKS,6055.T.T


Merging ESG score with market-capital of companies

In [45]:
df_market_cap = pd.merge(df_market_cap, df_market_caps, on='stock_ticker_symbol', left_index=False, right_index=False)
df_market_cap

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital
0,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,5.958660e+08
1,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.588731e+09
2,HNI Corp.,5.4,Negligible ESG Risk,Commercial Services,NYS,HNI,2.324179e+09
3,"Avnet, Inc.",5.6,Negligible ESG Risk,Technology Hardware,NAS,AVT,4.910356e+09
4,ACCO Brands Corp.,5.7,Negligible ESG Risk,Commercial Services,NYS,ACCO,4.781985e+08
...,...,...,...,...,...,...,...
954,"Halozyme Therapeutics, Inc.",19.9,Low ESG Risk,Pharmaceuticals,NAS,HALO,6.968251e+09
955,Matthews International Corp.,20.0,Low ESG Risk,Commercial Services,NAS,MATW,8.512784e+08
956,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI,9.148317e+07
957,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO,1.979536e+09


#### Market Capital to Euro

To make sense of the market-capital value we have to first pick a preferred currency and convert them all to it.
For this purpose we will be using Euro as the standard currency.


In [54]:
df_market_cap['market_capital_euro'] = df_market_cap['market_capital']

Exchange Rate (19.08.2024)

In [57]:
exchange_rate = {
    "yen_to_euro": 0.0058,
    "us_to_euro": 0.92,
    "pound_to_euro": 1.19
}

In [69]:
for i, row in enumerate(df_market_cap):
    # Yen
    if df_market_cap['stock_exchange'][i] == 'TKS':
        df_market_cap.at[i, 'market_capital_euro'] = df_market_cap['market_capital'][i] * exchange_rate['pound_to_euro']
    # Dollar
    if df_market_cap['stock_exchange'][i] == 'NYS':
        df_market_cap.at[i, 'market_capital_euro'] = df_market_cap['market_capital'][i] * exchange_rate['us_to_euro']
    if df_market_cap['stock_exchange'][i] == 'NAS':
        df_market_cap.at[i, 'market_capital_euro'] = df_market_cap['market_capital'][i] * exchange_rate['us_to_euro']
    # Pound
    if df_market_cap['stock_exchange'][i] == 'LON':
        df_market_cap.at[i, 'market_capital_euro'] = df_market_cap['market_capital'][i] * exchange_rate['pound_to_euro']

                        company_name  company_esg_score  \
0          Kimball Electronics, Inc.                4.5   
1                    Steelcase, Inc.                5.3   
2                          HNI Corp.                5.4   
3                        Avnet, Inc.                5.6   
4                  ACCO Brands Corp.                5.7   
..                               ...                ...   
954      Halozyme Therapeutics, Inc.               19.9   
955     Matthews International Corp.               20.0   
956  KVH Industries, Inc. (Delaware)               20.0   
957                NeoGenomics, Inc.               20.0   
958              Union Pacific Corp.               20.0   

    company_esg_score_group             industry stock_exchange  \
0       Negligible ESG Risk  Technology Hardware            NAS   
1       Negligible ESG Risk  Commercial Services            NYS   
2       Negligible ESG Risk  Commercial Services            NYS   
3       Negligible ESG 

In [72]:
df_market_cap.to_csv('../data/data_4_market_cap_euro.csv')
df_market_cap

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital,market_capital_euro
0,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,5.958660e+08,5.481967e+08
1,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.588731e+09,1.461633e+09
2,HNI Corp.,5.4,Negligible ESG Risk,Commercial Services,NYS,HNI,2.324179e+09,2.138245e+09
3,"Avnet, Inc.",5.6,Negligible ESG Risk,Technology Hardware,NAS,AVT,4.910356e+09,4.517528e+09
4,ACCO Brands Corp.,5.7,Negligible ESG Risk,Commercial Services,NYS,ACCO,4.781985e+08,4.399426e+08
...,...,...,...,...,...,...,...,...
954,"Halozyme Therapeutics, Inc.",19.9,Low ESG Risk,Pharmaceuticals,NAS,HALO,6.968251e+09,6.410791e+09
955,Matthews International Corp.,20.0,Low ESG Risk,Commercial Services,NAS,MATW,8.512784e+08,7.831761e+08
956,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI,9.148317e+07,8.416451e+07
957,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO,1.979536e+09,1.821173e+09


### Column normalization

Since ESG-Score and market-capital are on different scales, it's important to normalize them so they can be compared directly.

In [78]:
df_unscaled = df_market_cap.copy(deep=True)

In [79]:
# Normalize ESG-Score and market-capital
scaler = MinMaxScaler()
df_unscaled[['company_esg_score_scale', 'market_capital_scale']] = scaler.fit_transform(df_unscaled[['company_esg_score', 'market_capital_euro']])

In [80]:
df_unscaled

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital,market_capital_euro,company_esg_score_scale,market_capital_scale
0,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,5.958660e+08,5.481967e+08,0.000000,0.000159
1,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.588731e+09,1.461633e+09,0.051613,0.000442
2,HNI Corp.,5.4,Negligible ESG Risk,Commercial Services,NYS,HNI,2.324179e+09,2.138245e+09,0.058065,0.000652
3,"Avnet, Inc.",5.6,Negligible ESG Risk,Technology Hardware,NAS,AVT,4.910356e+09,4.517528e+09,0.070968,0.001389
4,ACCO Brands Corp.,5.7,Negligible ESG Risk,Commercial Services,NYS,ACCO,4.781985e+08,4.399426e+08,0.077419,0.000126
...,...,...,...,...,...,...,...,...,...,...
954,"Halozyme Therapeutics, Inc.",19.9,Low ESG Risk,Pharmaceuticals,NAS,HALO,6.968251e+09,6.410791e+09,0.993548,0.001975
955,Matthews International Corp.,20.0,Low ESG Risk,Commercial Services,NAS,MATW,8.512784e+08,7.831761e+08,1.000000,0.000232
956,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI,9.148317e+07,8.416451e+07,1.000000,0.000016
957,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO,1.979536e+09,1.821173e+09,1.000000,0.000554


To create a final score, we use the ESG score and the market capital score, applying the following weights: the ESG score is multiplied by a coefficient of 0.25, and the market capital score is multiplied by a coefficient of 0.75.

In [84]:
# Create final score
coefficient_esg_score = 0.25
coefficient_market_capital = 0.75
df_unscaled['score'] = df_unscaled['company_esg_score_scale'] * coefficient_esg_score + df_unscaled['market_capital_scale'] * coefficient_market_capital

# Sort by score
df_unscaled.sort_values(by='score', ascending=False)

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital,market_capital_euro,company_esg_score_scale,market_capital_scale,score
560,"Apple, Inc.",16.8,Low ESG Risk,Technology Hardware,NAS,AAPL,3.509669e+12,3.228895e+12,0.793548,1.000000,0.948387
282,Microsoft Corp.,14.2,Low ESG Risk,Software & Services,NAS,MSFT,3.296378e+12,3.032668e+12,0.625806,0.939227,0.860872
217,NVIDIA Corp.,13.2,Low ESG Risk,Semiconductors,NAS,NVDA,2.902354e+12,2.670165e+12,0.561290,0.826958,0.760541
821,Broadcom Inc.,18.9,Low ESG Risk,Semiconductors,NAS,AVGO,7.260681e+11,6.679827e+11,0.929032,0.206868,0.387409
584,"UnitedHealth Group, Inc.",17.0,Low ESG Risk,Healthcare,NYS,UNH,5.276384e+11,4.854273e+11,0.806452,0.150330,0.314360
...,...,...,...,...,...,...,...,...,...,...,...
1,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.588731e+09,1.461633e+09,0.051613,0.000442,0.013235
0,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,5.958660e+08,5.481967e+08,0.000000,0.000159,0.000120
594,Triton International Ltd.,17.1,Low ESG Risk,Commercial Services,NYS,TRTN.PRE,NaN,NaN,0.812903,NaN,NaN
645,National Rural Utilities Cooperative Finance C...,17.6,Low ESG Risk,Banks,NYS,NRUC,NaN,NaN,0.845161,NaN,NaN


In [85]:
df_unscaled.to_csv('../data/data_5_scaled.csv')